<h1>Predicting game prices</h1>
<h5 style="margin-left: 2rem">By: Shalev Hadar, Elad Ben-Haim</h5>
<p>Gmail/Email user: shaleveladhit@gmail.com</p>
<p>Steam Username: ShalevEladHit</p>
<p>Steam Password: ShalevElad123@</p>
<br/>
<br/>

<h4>Defining our problem</h4>
<table dir="rtl">
    <tr>
        <th>
            מה הבעיה
        </th>
        <th>
            תיאור הבעיה
        </th>
    </tr>
    <tr>
        <td>
        האם ניתן לחזות מחיר של משחק בעוד x זמן?
        </td>
        <td>
        מתי הכי ישתלם לקנות את המשחק בעתיד
        </td>
    </tr>
    <tr>
        <td>
        האם ניתן לחזור מתי למוכר הכי משתלם לעשות מבצע על המשחק?
        </td>
        <td>
        מתי הכי כדאי למוכר לעשות מבצע כדי להביא עוד שחקנים ולהמשיך למכור עם הרווח הגדול ביותר
        </td>
    </tr>
</table>
<br/>


<br/>

<h4>Importing</h6>

In [16]:
import requests
import bs4
from bs4 import BeautifulSoup
import random
import itertools
import re

<h4>Global functions</h4>

In [3]:
def get_html_response(url: str) -> requests.Response:
    return requests.get(url)

def get_response_as_beautiful_soup(htmlText: requests.Response) -> BeautifulSoup:
    return BeautifulSoup(htmlText.text, 'html.parser')

<br/>

<h4>Defining proxies for scraping</h4>

<h5>Get proxy list response html website</h5>
<h6>get the html as response object instead of getting the html again and again</h6>

In [4]:
# Get the html of the proxy list website
def get_proxy_list_html() -> requests.Response:
    # Website to get free proxies
    return get_html_response('https://free-proxy-list.net/')

proxies_response = get_proxy_list_html()

<h5>Scrape proxy ip addresses</h5>
<h6>gets the ip addresses as a list, shuffles them and returns an iterator to cycle through when making scrape requests</h6>

In [5]:
def get_proxy_list() -> itertools.cycle:
    soup = get_response_as_beautiful_soup(proxies_response)
    proxy_soup_list = soup.select('#list > div > div.table-responsive > div > table > tbody > tr')
    proxy_list = list(map(lambda i: i.select('td:nth-child(1)')[0].text + ':' + i.select('td:nth-child(2)')[0].text, proxy_soup_list))
    random.shuffle(proxy_list)
    return itertools.cycle(proxy_list)


<br/>

<h4>Scrape isThereAnyDeal website</h4>
<h6>Steps:</h6>
<ol>
<li>Crawl list of top 100 trending games</li>
<li>For each game:</li>
<ul>
    <li>get the game details from steam API using "appId" scraped either from PC Gaming Wiki or Steam Ladder links</li>
    <li>mine price Log history on isThereAnyDeal</li>
    <li>mine Number of sales of the game</li>
<ul>
</ol>

<h5>Get list of 100 top trending games</h5>

In [6]:
def get_is_there_any_deal_games_response() -> requests.Response:
    filteredUrl = 'https://isthereanydeal.com/?by=trending:desc#/filter:&releaseyear/2010/2021,&pl/windows,&drm/steam,steam,-dlc,-type/6,-type/8,-type/7;/options:all'
    return get_html_response(filteredUrl)
is_there_any_deal_games_response = get_is_there_any_deal_games_response()

In [7]:
def get_game_links():
    soup = get_response_as_beautiful_soup(is_there_any_deal_games_response)
    print(soup.select(""))
    

In [36]:
def get_game_details_response():
    return get_html_response("https://isthereanydeal.com/game/deadbydaylight/history/#/chart:low")

game_details_response = get_game_details_response()

In [37]:
def get_game_details():
    soup = get_response_as_beautiful_soup(game_details_response)
    return soup.select("#pageContainer > script:nth-child(9)")

gamedetails = str(get_game_details()[0])
m = re.findall('JSON\.stringify\((.*?)\)', gamedetails)
# print(m)

['[{"x":1461367284000,"y":10,"duration":72.71075231481481,"bucket":["4","April"],"shop":"steam","color":"#9ffc3a"},{"x":1464973310000,"y":10,"duration":10.98943287037037,"bucket":["6","June"],"shop":"humblestore","color":"#ff3e1b"},{"x":1466571339000,"y":10,"duration":12.482569444444444,"bucket":["6","June"],"shop":"humblestore","color":"#ff3e1b"},{"x":1471544130000,"y":15,"duration":4.000034722222222,"bucket":["8","August"],"shop":"steam","color":"#9ffc3a"},{"x":1477678529000,"y":30,"duration":4.062581018518519,"bucket":["10","October"],"shop":"steam","color":"#9ffc3a"},{"x":1479930415000,"y":30,"duration":5.999143518518519,"bucket":["11","November"],"shop":"steam","color":"#9ffc3a"},{"x":1481831134000,"y":30,"duration":4.041643518518518,"bucket":["12","December"],"shop":"steam","color":"#9ffc3a"},{"x":1482441359000,"y":30,"duration":10.979224537037037,"bucket":["12","December"],"shop":"steam","color":"#9ffc3a"},{"x":1484244386000,"y":30,"duration":13.993252314814814,"bucket":["1","Ja

<h3>Used Resources</h3>
<dl>
    <dt>Scraping</dt>
    <dd>
        <a href="https://isthereanydeal.com/game/reddeadredemptionii/info/">
            <b>Is-There-Any-Deal website</b> For scraping cost history and more financial details
        </a>
    </dd>
    <dd>
        <a href="https://www.geeksforgeeks.org/web-scraping-without-getting-blocked/">
            <b>Using Proxies to avoiding getting blocked</b>
        </a>
    </dd>
    <dd>
        <a href="https://wiki.teamfortress.com/wiki/User:RJackson/StorefrontAPI#App_info">
            <b>Steam StoreFront API</b> Limited to 100,000 requests per day, and no more than 10 per second
        </a>
    </dd>
    <dd>
        <a href="https://store.steampowered.com/api/appdetails">Steam API for Game Metadata - https://store.steampowered.com/api/appdetails?appids=1091500</a>
    </dd>
</dl>